In [1]:
import pandas as pd
from IPython.display import display
import numpy as np


In [2]:
order_items = pd.read_csv("CSV2/order_items_cleaned.csv")
products = pd.read_csv("CSV2/products_cleaned.csv")

# 2. Filtriamo per escludere gli ordini annullati (teniamo solo vendite reali)
valid_sales = order_items[(order_items['items_status'] != 'Cancelled') & (order_items['items_status'] != 'Returned')]


In [3]:
valid_sales = valid_sales.merge(
    products[['product_id', 'category', 'brand', 'retail_price', 'cost']],
    on='product_id',
    how='left'
)

In [4]:
# margine per riga
valid_sales["margin"] = valid_sales["sale_price"] - valid_sales["cost"]
valid_sales["margin_pct"] = np.where(valid_sales["sale_price"] != 0, valid_sales["margin"] / valid_sales["sale_price"], np.nan)

# aggregazione per brand
margini_brand = (
    valid_sales.groupby("brand", dropna=False)
      .agg(
          n_orders=("order_id", "nunique"),
          n_items=("order_id", "size"),
          revenue=("sale_price", "sum"),
          cost=("cost", "sum"),
          margin=("margin", "sum"),
      )
      .assign(margin_pct=lambda x: np.where(x["revenue"] != 0, x["margin"] / x["revenue"], np.nan))
      .sort_values("margin", ascending=False)
      .reset_index()
)

margini_brand

,brand,n_orders,n_items,revenue,cost,margin,margin_pct
0,Calvin Klein,2487,2514,161524.220200,75752.01,85772.210200,0.531018
1,Diesel,1075,1083,146986.149899,74228.40,72757.749899,0.494997
2,Carhartt,1904,1917,132634.079054,61728.61,70905.469054,0.534595
3,7 For All Mankind,873,875,134479.490429,70109.00,64370.490429,0.478664
4,True Religion,663,663,133686.449602,69714.40,63972.049602,0.478523
...,...,...,...,...,...,...,...
2741,Tabi Socks,1,1,5.990000,2.57,3.420000,0.570952
2742,Eros,2,2,5.900000,2.78,3.120000,0.528814
2743,ZOCKS,1,1,6.500000,3.98,2.520000,0.387692
2744,Made in USA,5,5,2.450000,0.90,1.550000,0.632653


In [5]:
# 1. Calcolo del margine per singola riga (se non lo hai già fatto)
valid_sales["margin"] = valid_sales["sale_price"] - valid_sales["cost"]

# 2. Aggregazione per categoria
margini_categoria = (
    valid_sales.groupby("category", dropna=False)
      .agg(
          n_items=("order_id", "size"),      # Numero di pezzi venduti
          revenue=("sale_price", "sum"),     # Fatturato totale
          total_cost=("cost", "sum"),        # Costo totale
          total_margin=("margin", "sum")     # Margine totale (Euro)
      )
      .assign(
          # Margine percentuale calcolato sul totale della categoria
          margin_pct=lambda x: np.where(x["revenue"] != 0, x["total_margin"] / x["revenue"], np.nan)
      )
      .sort_values("margin_pct", ascending=False)
      .reset_index()
)

margini_categoria.head()

,category,n_items,revenue,total_cost,total_margin,margin_pct
0,Blazers & Jackets,2412,225402.370448,85617.91,139784.460448,0.620155
1,Skirts,1514,76122.600183,30361.95,45760.650183,0.601144
2,Accessories,7332,307874.089772,123445.15,184428.939772,0.599040
3,Suits & Sport Coats,3799,475956.689150,191006.74,284949.949150,0.598689
4,Socks & Hosiery,2895,48639.309913,19584.16,29055.149913,0.597359
